In [49]:
%load_ext autoreload
%autoreload 2
import hippo
import mrich
from mrich import print
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
animal = hippo.HIPPO("RdRp", "../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite")

 Creating HIPPO animal

name = RdRp

db_path = ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite!

 Success  Initialised animal HIPPO("RdRp")!

## Export SMILES by pockets

In [20]:
TAGS = [
    "Green Site/RNA tunnel",
    "Primer grip/N-pocket",
    "Cavity B",
    "Thumb Site 2",
]

In [22]:
POSES = {}
for tag in TAGS:
    poses = animal.poses(tag=f"{tag} merge")
    print(poses)
    POSES[tag] = dict(merge_poses=poses)

poses tagged "Green Site/RNA tunnel merge": {P × 172966}

poses tagged "Primer grip/N-pocket merge": {P × 28894}

poses tagged "Cavity B merge": {P × 452}

poses tagged "Thumb Site 2 merge": {P × 4892}

In [27]:
%%time
for k,v in POSES.items():
    print(k)
    poses = v["merge_poses"]
    compounds = poses.compounds
    POSES[k]["merge_compounds"] = compounds
    POSES[k]["merge_smiles"] = compounds.smiles

Green Site/RNA tunnel

Primer grip/N-pocket

Cavity B

Thumb Site 2

CPU times: user 390 ms, sys: 1.82 s, total: 2.21 s
Wall time: 18 s


In [28]:
%%time
for k,v in POSES.items():
    print(k)
    poses = v["merge_poses"]
    compounds = v["merge_compounds"]
    elabs = compounds.elabs
    POSES[k]["merge_smiles"] = compounds.smiles

{C × 3234}

In [38]:
compounds = POSES[TAGS[1]]["merge_compounds"]
compounds

{C × 11534}

In [39]:
elabs = compounds.elabs
elabs

{C × 20993}

In [40]:
elabs.bases

{C × 7}

In [53]:
%%time
dfs = []
for k,v in POSES.items():
    print(k)
    compounds = v["merge_compounds"]
    elabs = compounds.elabs
    print(elabs)
    df = elabs.get_df(alias=False, bases=True, debug=True)
    print(len(df))
    df["site"] = k
    df.rename(columns={"bases":"scaffold"}, inplace=True)
    scaffolds = elabs.bases
    lookup = animal.db.get_compound_id_smiles_dict(scaffolds)
    df["scaffold"] = df["scaffold"].apply(lambda x: lookup[list(x)[0]])
    dfs.append(df)
df = pd.concat(dfs)
df.head()

Green Site/RNA tunnel

{C × 148292}

DEBUG: querying...

Output()

DEBUG: adding scaffold column(s)

...

Output()

DEBUG: adding scaffold column(s)

3234

CPU times: user 1.87 s, sys: 332 ms, total: 2.21 s
Wall time: 5.8 s


,smiles,scaffold,site
id,,,
241786,BrCC(NC1CCCc2c(Br)ccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
241784,BrCC(NC1CCCc2cc(Br)cnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
241785,BrCC(NC1CCCc2ccc(Br)nc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
240771,BrCC(NC1CCCc2cccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
240575,BrCC(c1cccnc1N1CCCC1)N1C2CCC1c1ncccc12,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel


In [54]:
df

,smiles,scaffold,site
id,,,
241786,BrCC(NC1CCCc2c(Br)ccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
241784,BrCC(NC1CCCc2cc(Br)cnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
241785,BrCC(NC1CCCc2ccc(Br)nc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
240771,BrCC(NC1CCCc2cccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
240575,BrCC(c1cccnc1N1CCCC1)N1C2CCC1c1ncccc12,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel
...,...,...,...
77222,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2C(CC3CC3)C(=O)N2C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2
77223,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2C(CC(F)F)C(=O)N2C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2
77224,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2N(C3CC3)C(=O)C2(C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2


In [55]:
df.to_csv("Flavi_RdRp_elabs.csv")